# **LINEAR REGRESSION BLOCK START**

In [1]:
# Import dependencies

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [52]:
df = pd.read_csv("Hitters_Adjusted_Salary.csv")

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15023 entries, 0 to 15022
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  15023 non-null  int64  
 1   yearID      15023 non-null  int64  
 2   playerID    15023 non-null  object 
 3   salary      15023 non-null  int64  
 4   ADJ Salary  15023 non-null  float64
 5   GS          15023 non-null  float64
 6   InnOuts     15023 non-null  float64
 7   PO          15023 non-null  int64  
 8   A           15023 non-null  int64  
 9   E           15023 non-null  float64
 10  DP          15023 non-null  int64  
 11  teamID      15023 non-null  object 
 12  lgID        15023 non-null  object 
 13  G           15023 non-null  int64  
 14  AB          15023 non-null  int64  
 15  R           15023 non-null  int64  
 16  H           15023 non-null  int64  
 17  2B          15023 non-null  int64  
 18  3B          15023 non-null  int64  
 19  HR          15023 non-nul

In [36]:
df.sample(25)

,Unnamed: 0,yearID,playerID,salary,ADJ Salary,GS,InnOuts,PO,A,E,DP,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
6597,6597,1999,boonebr01,2900000,4.505113e+06,144.0,3890.0,270,424,13.0,78,ATL,NL,152,608,102,153,38,1,20,63.0,14.0,9.0,47,112.0,0.0,5.0,9.0,2.0,11.0
5104,5104,1996,flahejo01,275000,4.536203e+05,107.0,2891.0,714,42,10.0,4,SDN,NL,72,264,22,80,12,0,9,41.0,2.0,3.0,9,36.0,1.0,2.0,1.0,3.0,8.0
14670,14670,2016,holtty01,508000,5.478006e+05,33.0,1139.0,94,1,1.0,0,CIN,NL,106,179,21,42,5,3,0,13.0,4.0,3.0,23,48.0,0.0,2.0,3.0,1.0,6.0
3002,3002,1992,dawsoan01,3300000,6.087500e+06,137.0,3547.0,223,11,2.0,3,CHN,NL,143,542,60,150,27,2,22,90.0,6.0,2.0,30,70.0,8.0,4.0,0.0,6.0,13.0
1389,1389,1988,riverlu01,70000,1.531426e+05,101.0,2772.0,160,301,18.0,69,MON,NL,123,371,35,83,17,3,4,30.0,3.0,4.0,24,69.0,4.0,1.0,3.0,3.0,9.0
7203,7203,2000,thomafr04,7100000,1.067107e+07,30.0,775.0,267,15,1.0,38,CHA,AL,159,582,115,191,44,0,43,143.0,1.0,3.0,112,94.0,18.0,5.0,0.0,8.0,13.0
6287,6287,1998,cirilje01,1500000,2.381696e+06,153.0,4078.0,149,353,11.0,53,MIL,NL,156,604,97,194,31,1,14,68.0,10.0,4.0,79,88.0,3.0,4.0,5.0,2.0,26.0
2064,2064,1990,bicheda01,100000,1.980191e+05,86.0,2423.0,183,12,7.0,5,CAL,AL,109,349,40,89,15,1,15,53.0,5.0,2.0,16,79.0,1.0,3.0,1.0,2.0,9.0
7743,7743,2001,taubeed01,1800000,2.630490e+06,28.0,743.0,212,3,3.0,0,CLE,AL,52,116,16,29,2,1,3,11.0,0.0,0.0,10,19.0,1.0,1.0,1.0,0.0,3.0
5489,5489,1997,mcgrifr01,5500000,8.868913e+06,148.0,3791.0,1191,96,13.0,111,ATL,NL,152,564,77,156,25,1,22,97.0,5.0,0.0,68,112.0,4.0,4.0,0.0,5.0,22.0


In [53]:
df = df[["yearID", "playerID", "teamID", "H", "R", "RBI", "AB", "ADJ Salary"]]

In [54]:
df

,yearID,playerID,teamID,H,R,RBI,AB,ADJ Salary
0,1985,benedbr01,ATL,42,12,20.0,208,1.310892e+06
1,1985,ceronri01,ATL,61,15,25.0,282,1.503317e+06
2,1985,chambch01,ATL,40,16,21.0,170,1.924245e+06
3,1985,harpete01,ATL,130,58,72.0,492,6.013267e+05
4,1985,hornebo01,ATL,129,61,89.0,483,3.607960e+06
...,...,...,...,...,...,...,...,...
15018,2016,reverbe01,WAS,76,44,24.0,350,6.739673e+06
15019,2016,robincl01,WAS,46,16,26.0,196,5.768082e+05
15020,2016,taylomi02,WAS,51,28,16.0,221,5.650542e+05
15021,2016,werthja01,WAS,128,84,69.0,525,2.343639e+07


In [55]:
df["ADJ Salary"] = df["ADJ Salary"].astype("int").round()

In [67]:
df = df.drop_duplicates(subset=["yearID", "playerID"]).reset_index(drop=True)
df

,yearID,playerID,teamID,H,R,RBI,AB,ADJ Salary
0,1985,benedbr01,ATL,42,12,20.0,208,1310892
1,1985,ceronri01,ATL,61,15,25.0,282,1503316
2,1985,chambch01,ATL,40,16,21.0,170,1924245
3,1985,harpete01,ATL,130,58,72.0,492,601326
4,1985,hornebo01,ATL,129,61,89.0,483,3607960
...,...,...,...,...,...,...,...,...
13601,2016,reverbe01,WAS,76,44,24.0,350,6739673
13602,2016,robincl01,WAS,46,16,26.0,196,576808
13603,2016,taylomi02,WAS,51,28,16.0,221,565054
13604,2016,werthja01,WAS,128,84,69.0,525,23436394


In [70]:
df.duplicated(subset=["yearID", "playerID", "teamID"]).value_counts()

False    13606
dtype: int64

In [71]:
df.to_csv("cleaned_hitter.csv")

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13606 entries, 0 to 13605
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   yearID      13606 non-null  int64  
 1   playerID    13606 non-null  object 
 2   teamID      13606 non-null  object 
 3   H           13606 non-null  int64  
 4   R           13606 non-null  int64  
 5   RBI         13606 non-null  float64
 6   AB          13606 non-null  int64  
 7   ADJ Salary  13606 non-null  int32  
dtypes: float64(1), int32(1), int64(4), object(2)
memory usage: 797.4+ KB


In [74]:
df["yearID"].value_counts()

1999    512
1995    503
1998    494
1993    492
1996    478
1997    466
1994    465
1990    451
2001    438
2000    429
2014    428
2003    427
2002    425
2004    423
2012    421
2015    416
2005    415
2016    415
2008    414
2011    414
1992    413
2007    413
2013    410
2006    408
2010    408
1986    404
2009    404
1989    400
1988    379
1991    373
1987    358
1985    310
Name: yearID, dtype: int64

In [ ]:
df["playerID"].value_counts()

In [ ]:
df.groupby(["yearID", "playerID"])

In [59]:
grouped_df = df.groupby(["playerID"]).count()
years_in_MLB = grouped_df["yearID"]
years_in_MLB

playerID
abbotje01     4
abbotku01    10
abercre01     1
abernbr01     3
abnersh01     6
             ..
zuberjo01     1
zuletju01     1
zuninmi01     2
zupcibo01     3
zuvelpa01     1
Name: yearID, Length: 2468, dtype: int64

In [48]:
df["years_in_MLB"] = years_in_MLB

In [66]:
mask = df.groupby(["yearID", "playerID"]).sum().duplicated()

df = df[mask]

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'

In [ ]:
def load_dataset():
    """ Reads dataset csv and returns pandas dataframe """
    
    filepath = "Hitters_Adjusted_Salary.csv"

    df = pd.read_csv(filepath, encoding="utf-8", low_memory=False)
    
    return df

In [ ]:
def clean_dataset(a_df):
    """ Returns deduped, na-dropped, index-reset dataframe """    
    
    a_df = a_df.drop_duplicates()   
        
    a_df = a_df.dropna()
    
    a_df = a_df.drop(["Unnamed: 0", "yearID", "playerID", "teamID", "lgID", "salary", "PO", "A", "E", "DP"], axis=1)
        
    a_df = a_df.reset_index(drop=True)
    
    return a_df   

In [ ]:
def rename_columns(a_df):
    """ Returns dataframe with meaningful column names """    
    
    abbr_dict = {"GS": "games_started", "InnOuts": "inning_outs", "G": "games_played", "AB": "at_bats",\
                "R": "runs", "H": "hits", "2B": "doubles", "3B": "triples", "HR": "home_runs",\
                "RBI": "runs_batted_in", "SB": "stolen_bases", "CS": "caught_stealing", "BB": "base_on_balls",\
                "SO": "strike_outs", "IBB": "intentional_walks", "HBP": "hit_by_pitch", "SH": "sacrifice_hits",\
                "SF": "sacrifice_flies", "GIDP": "ground_into_double_play"}
    
    a_df = a_df.rename(columns=abbr_dict)
    
    return a_df   

In [ ]:
def examine_dataset(a_df):
    """ Provides summary info and visualizations of dataset """
    
    print(a_df.info())
    
    a_df.hist(figsize = (15, 15))  
    

In [ ]:
def round_salaries(a_df):
    """ Returns dataframe with salary column made into int and rounded """

    a_df["ADJ Salary"] = a_df["ADJ Salary"].astype("int").round()

    return a_df

In [ ]:
def run_regression(a_df, salary_col_name="ADJ Salary"):
    """ Runs linear regression on dataframe, prints model scores """

    # Assign X and y

    X = a_df.drop([salary_col_name], axis=1)
    y = a_df[salary_col_name]
    
    # Split the data into X_train, X_test, y_train, y_test

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

    # Create a scaler to standardize the data

    scaler = StandardScaler()

    # Train the scaler with the X_train data.

    scaler.fit(X_train)

    # Transform X_train and X_test.

    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model = LinearRegression().fit(X_train_scaled, y_train)
    
    predicted = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, predicted)
    r2 = r2_score(y_test, predicted)
    
    print(f"MSE: {mse}")
    print(f"R2: {r2}") 
    
    # Score the model

    print(f"Linear Regression Training Data Score: {model.score(X_train_scaled, y_train)}")
    print(f"Linear Regression Testing Data Score: {model.score(X_test_scaled, y_test)}")  
    
    plt.bar(np.arange(len(model.coef_)), model.coef_)
    plt.title(f'Linear Regression coefficient plot')
    plt.show()  

In [ ]:
def run_LASSO(a_df):
    """ Runs LASSO regression on dataframe, prints model scores """

    # Assign X and y

    X = a_df.drop(["log_of_salary"], axis=1)
    y = a_df["log_of_salary"]
    
    # Split the data into X_train, X_test, y_train, y_test

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

    # Create a scaler to standardize the data

    scaler = StandardScaler()

    # Train the scaler with the X_train data.

    scaler.fit(X_train)

    # Transform X_train and X_test.

    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    lasso_reg = Lasso().fit(X_train, y_train)

    predicted = lasso_reg.predict(X_test_scaled)
    mse = mean_squared_error(y_test, predicted)
    r2 = r2_score(y_test, predicted)
    
    print(f"MSE: {mse}")
    print(f"R2: {r2}") 

    # Score the model

    print(f"LASSO Regression Training Data Score: {lasso_reg.score(X_train_scaled, y_train)}")
    print(f"LASSO Regression Testing Data Score: {lasso_reg.score(X_test_scaled, y_test)}")
    
    plt.bar(np.arange(len(lasso_reg.coef_)), lasso_reg.coef_)
    plt.title(f'LASSO Regression coefficient plot')
    plt.show()  
    
    sel = SelectFromModel(lasso_reg)
    sel.fit(X_train_scaled, y_train)
    SelectFromModel(estimator=Lasso())
    
    X_selected_train, X_selected_test, y_train, y_test = train_test_split(sel.transform(X), y, random_state=1)
    
    scaler = StandardScaler().fit(X_selected_train)
    
    X_selected_train_scaled = scaler.transform(X_selected_train)
    X_selected_test_scaled = scaler.transform(X_selected_test)

    new_lasso_reg = LinearRegression().fit(X_selected_train_scaled, y_train)
    print(f"New linear regression score: {new_lasso_reg.score(X_selected_test_scaled, y_test)}")

In [ ]:
def run_Ridge(a_df):
    """ Runs Ridge regression on dataframe, prints model scores """

    # Assign X and y

    X = a_df.drop(["log_of_salary"], axis=1)
    y = a_df["log_of_salary"]
    
    # Split the data into X_train, X_test, y_train, y_test

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

    # Create a scaler to standardize the data

    scaler = StandardScaler()

    # Train the scaler with the X_train data.

    scaler.fit(X_train)

    # Transform X_train and X_test.

    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    ridge_reg = Ridge().fit(X_train, y_train)

    predicted = ridge_reg.predict(X_test_scaled)
    mse = mean_squared_error(y_test, predicted)
    r2 = r2_score(y_test, predicted)
    
    print(f"MSE: {mse}")
    print(f"R2: {r2}") 

    # Score the model

    print(f"Ridge Regression Training Data Score: {ridge_reg.score(X_train_scaled, y_train)}")
    print(f"Ridge Regression Testing Data Score: {ridge_reg.score(X_test_scaled, y_test)}")

    plt.bar(np.arange(len(ridge_reg.coef_)), ridge_reg.coef_)
    plt.title(f'Ridge Regression coefficient plot')
    plt.show()     
    
    sel = SelectFromModel(ridge_reg)
    sel.fit(X_train_scaled, y_train)
    SelectFromModel(estimator=Ridge())
    
    X_selected_train, X_selected_test, y_train, y_test = train_test_split(sel.transform(X), y, random_state=1)
    
    scaler = StandardScaler().fit(X_selected_train)
    
    X_selected_train_scaled = scaler.transform(X_selected_train)
    X_selected_test_scaled = scaler.transform(X_selected_test)

    new_ridge_reg = LinearRegression().fit(X_selected_train_scaled, y_train)
    print(f"New linear regression score: {new_ridge_reg.score(X_selected_test_scaled, y_test)}")

In [ ]:
def run_ElasticNet(a_df):
    """ Runs ElasticNet regression on dataframe, prints model scores """

    # Assign X and y

    X = a_df.drop(["log_of_salary"], axis=1)
    y = a_df["log_of_salary"]
    
    # Split the data into X_train, X_test, y_train, y_test

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

    # Create a scaler to standardize the data

    scaler = StandardScaler()

    # Train the scaler with the X_train data.

    scaler.fit(X_train)

    # Transform X_train and X_test.

    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    elasticnet_reg = ElasticNet().fit(X_train, y_train)
      
    predicted = elasticnet_reg.predict(X_test_scaled)
    mse = mean_squared_error(y_test, predicted)
    r2 = r2_score(y_test, predicted)
    
    print(f"MSE: {mse}")
    print(f"R2: {r2}") 

    # Score the model

    print(f"ElasticNet Regression Training Data Score: {elasticnet_reg.score(X_train_scaled, y_train)}")
    print(f"ElasticNet Regression Testing Data Score: {elasticnet_reg.score(X_test_scaled, y_test)}")
    
    plt.bar(np.arange(len(elasticnet_reg.coef_)), elasticnet_reg.coef_)
    plt.title(f'ElasticNet Regression coefficient plot')
    plt.show()  
    
    sel = SelectFromModel(elasticnet_reg)
    sel.fit(X_train_scaled, y_train)
    SelectFromModel(estimator=ElasticNet())
    
    X_selected_train, X_selected_test, y_train, y_test = train_test_split(sel.transform(X), y, random_state=1)
    
    scaler = StandardScaler().fit(X_selected_train)
    
    X_selected_train_scaled = scaler.transform(X_selected_train)
    X_selected_test_scaled = scaler.transform(X_selected_test)

    new_elasticnet_reg = LinearRegression().fit(X_selected_train_scaled, y_train)
    print(f"New linear regression score: {new_elasticnet_reg.score(X_selected_test_scaled, y_test)}")

In [ ]:
loaded_df = load_dataset()

In [ ]:
loaded_df["ADJ Salary"] = loaded_df["ADJ Salary"].astype(int)

loaded_df = loaded_df.loc[~(loaded_df["ADJ Salary"] <= 0), :]

In [ ]:
clean_df = clean_dataset(loaded_df)

In [ ]:
clean_df = rename_columns(clean_df)

In [ ]:
clean_df.head(10)

In [ ]:
examine_dataset(clean_df)

In [ ]:
clean_df = round_salaries(clean_df)

In [ ]:
clean_df["log_of_salary"] = np.log(clean_df["ADJ Salary"])

In [ ]:
clean_df["log_of_salary"].describe()

In [ ]:
run_regression(clean_df, "ADJ Salary")

In [ ]:
run_regression(clean_df, "log_of_salary")

In [ ]:
run_LASSO(clean_df)

In [ ]:
run_Ridge(clean_df)

In [ ]:
run_ElasticNet(clean_df)

# Results:

- Using the logarithm of the ADJ Salary column values improved the regression score
    - I had to eliminate any salary values <= 0 to make this work
- Ridge regression achieved the best score: 0.7291495360671353

# To Do:

- Add / remove features from the dataset?


# **LINEAR REGRESSION BLOCK END**

# **PCA START BLOCK**

In [ ]:
clean_df.head(10)

In [ ]:
clean_df.columns

In [ ]:
#import dependencies
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
# Drop ADJ Salary column
clean_df2= clean_df.drop(["ADJ Salary"], axis=1)
clean_df2.head()

In [ ]:
clean_df2.shape

In [ ]:
# Standardize the dataset so that columns that contain larger values do not influence the outcome more than columns with smaller values.
clean_scaled = StandardScaler().fit_transform(clean_df2)
print(clean_scaled)

In [ ]:
# Perform dimensionality reduction with PCA preserving 90% of the explained variance ( n_components=0.90)
# Initialize PCA model
pca = PCA(n_components=0.90)

# Get two principal components for the iris data.
clean_pca = pca.fit_transform(clean_scaled)

In [ ]:
clean_pca.shape

In [ ]:
clean_df_pca = pd.DataFrame(data=clean_pca)
clean_df_pca

In [ ]:
# Transform PCA data to a DataFrame
clean_df_pca = pd.DataFrame(data=clean_pca, columns=["principal component 1", "principal component 2",
                                                                "principal component 3","principal component 4",
                                                                 "principal component 5","principal component 6",
                                                                 "principal component 7","principal component 8"])
clean_df_pca.head()

In [ ]:
clean_df_pca.columns

In [ ]:
# Fetch the explained variance
pca.explained_variance_ratio_

In [ ]:
# Further reduce the dataset dimensions with t-SNE

# Initialize t-SNE model
tsne = TSNE(learning_rate = 50)

# Reduce dimensions
tsne_features = tsne.fit_transform(clean_pca)

# The dataset has 2 columns
tsne_features.shape

In [ ]:
# Prepare to plot the dataset

# The first column of transformed features
clean_df2["x"] = tsne_features[:,0]

# The second column of transformed features
clean_df2["y"] = tsne_features[:,1]

# Visualize the clusters
plt.scatter(clean_df2["x"],clean_df2["y"])
plt.show()

In [ ]:
labels = clean_df["ADJ Salary"]
labels.value_counts()

In [ ]:
# Visualize the clusters with color
plt.scatter(clean_df2["x"],clean_df2["y"], c=labels)
plt.show()

# Perform a Cluster Analysis with K-means

In [ ]:
#Windows KMeans bug fix. 
import os
os.environ["OMP_NUM_THREADS"] = '1'

In [ ]:
# Finding the best value for k using the Elbow Curve
inertia = []
k = list(range(1, 9))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(clean_df_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow

In [ ]:
# Plot the elbow curve to find the best candidate(s) for k
plt.plot(df_elbow['k'], df_elbow['inertia'])
plt.xticks(list(range(11)))
plt.title('Elbow Curve')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
# If possible, determine where the elbow of the plot is, and at which value of k it appears.
# Create a function called `get_clusters(k, data)` that finds the `k` clusters using K-Means on `data`. The function should return a DataFrame copy of `Data` that should include a new column containing the clusters found.

def get_clusters(k, data):
    # Initialize the K-Means model
    model = KMeans(n_clusters=k, random_state=0)

    # Train the model
    model.fit(data)

    # Predict clusters
    predictions = model.predict(data)

    # Create return DataFrame with predicted clusters
    data["class"] = model.labels_

    return data

In [ ]:
clusters = get_clusters(2, clean_df_pca)
clusters

In [ ]:
def show_clusters(df):
    plt.scatter(df['principal component 1'], df['principal component 2'], c=df['class'])
    plt.xlabel('principal component 1')
    plt.ylabel('principal component 2')
    plt.title("ADJ Salary clusters")
    plt.show()

In [ ]:
show_clusters(clusters)

# **PCA END BLOCK**